In [14]:
import hashlib
import sympy
import binascii
from ecpy.curves import Curve, Point

Leemos la Clave publica de la conexion a partir del fichero obtenido

In [15]:
with open('../files/Wikipedia/subjectPublicKey.txt') as f:
    subjectPK = f.read()
Qx = int(subjectPK[2:66], 16)
Qy = int(subjectPK[66:], 16)

Cojemos los demas datos de la curva

In [16]:
cv = Curve.get_curve('secp256r1')
p = cv.field
n = cv.order
q = n
b = cv.b
P = cv.generator

Q = Point(Qx, Qy, cv)


#### a) Comprovamos que el orden de la curva es un numero primo

In [24]:
sympy.isprime(p)

True

#### b) comprovamos que el punto de wikipedia es realmente un punto de la curva

In [26]:
r = Q.y ** 2
r2 = Q.x ** 3 - 3 * Q.x + b
r % p == r2 % p

True

#### c) Comprovamos el orden del punto

In [19]:
(Q * q).is_infinity

True

vemos que el orden del punto es q

#### d) Comprovamos que la firma es correcta

Leemos la firma de wikipedia del fichero extraido de la conexion

In [20]:
with open('../files/Wikipedia/signature.txt') as f:
    signature = f.read()
f1 = int(signature[8:74], 16)
f2 = int(signature[78:], 16)

In [21]:
pre = "20"*64 + "544C5320312E332C2073657276657220436572746966696361746556657269667900".lower()

Leemos el mensage original del fichero extraido de la conexion

In [22]:
with open('../files/Wikipedia/mensaje.bin', 'rb') as f:
    message = f.read()
    h = hashlib.sha384(message).hexdigest()
    m = int(hashlib.sha256(bytes(bytearray.fromhex(pre + h))).hexdigest(), 16)

Comprovamos que la firma es valida

In [27]:
f2i = sympy.mod_inverse(f2, n)
w1 = (m * f2i) % n
w2 = (f1 * f2i) % n

E = (w1 * P) + (w2 * Q)

if (E.x % n) == f1:
    print("Firma correcta")
else:
    print("Firma incorrecta")


Firma correcta
